<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-Big-Data-with-HyperSpy" data-toc-modified-id="Introduction-to-Big-Data-with-HyperSpy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to Big Data with HyperSpy</a></span><ul class="toc-item"><li><span><a href="#Loading-and-plotting-a-large-spectrum-image-lazily" data-toc-modified-id="Loading-and-plotting-a-large-spectrum-image-lazily-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Loading and plotting a large spectrum image lazily</a></span><ul class="toc-item"><li><span><a href="#Creating-a-&quot;navigator&quot;-for-the-two-views-of-the-dataset" data-toc-modified-id="Creating-a-&quot;navigator&quot;-for-the-two-views-of-the-dataset-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Creating a "navigator" for the two views of the dataset</a></span></li></ul></li><li><span><a href="#ROI-in-navigation-dimension" data-toc-modified-id="ROI-in-navigation-dimension-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>ROI in navigation dimension</a></span></li><li><span><a href="#Decomposition" data-toc-modified-id="Decomposition-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Decomposition</a></span></li><li><span><a href="#Rebinning" data-toc-modified-id="Rebinning-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Rebinning</a></span></li></ul></li><li><span><a href="#Curve-fitting" data-toc-modified-id="Curve-fitting-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Curve fitting</a></span></li></ul></div>

# Introduction to Big Data with HyperSpy

## Loading and plotting a large spectrum image lazily

First we setup the matplotlib backend and import hyperspy

In [ ]:
%matplotlib qt

In [ ]:
import hyperspy.api as hs

Let's start by setting the ``logging level`` to "INFO" using the preferences GUI as below. Once done, click ``Save`` and restart the kernel.

In [8]:
hs.preferences.gui()

For this tutorial we are going to start by loading a large spectrum image lazily  that won't fit in the RAM of a standard laptop computer in 2018, so notice the ``lazy`` keyword and don't forget to set it to ``True``.

In [9]:
s = hs.load("lazy/lazy_demo_data_hyperspy_chunking.hspy", lazy=True)

Let's check what sort of object we have stored in the ``s`` variable

In [10]:
print(s)

<LazySignal1D, title: Stack of  (0,), dimensions: (512, 512|4096)>


This is a spectrum image with *4096* spectral channels and *512x512* image dimensions.

Let's calculate the size of the data in GB:

In [11]:
print(s.data.nbytes / 1e9)

8.589934592


That's more than *8 GB* of data. However, loading it took no time. That's because HyperSpy didn't load the data yet. It'll do it on demand. More on this later.

Let's plot the spectrum image. Usually we would call simply:

```python
s.plot()
```

And a navigator would appear. However, in this case, computing the navigator would take too long. Therefore, we start by navigating the dataset "blindly" using sliders.

Notice the ``Continous update`` checkbox in the ipywidgets (in you have the ``hyperspy_gui_ipywidgets`` installed and well configured). Unchecking it won't update the signal figure until you release the slider, what is handy when dealing with large datasets.

In [12]:
s.plot(navigator="slider")

If we were not working in lazy mode we could easily explore this dataset as an image stack by transposing and plotting it as follows:

```python
s.T.plot(navigator="slider")
```

However, executing the code above with the current file risks saturating your computer memory. This is because of the way the data is chunked in the file. Let's have a look at the data chunks:

In [13]:
print(s.data)

dask.array<array, shape=(512, 512, 4096), dtype=int64, chunksize=(4, 4, 4096)>


This means that the data is stored in the file as *4x4x4096* blocks with the biggest (4096) dimension corresponding to the spectral dimension. In other words, the chunks are *4 px x 4 px* spectrum images. This chunking is good if we want to perform operations that operate on indiviual spectra. However, it is a terrible chunking configuration for e.g. slicing the spectrum image as this operation would require loading the whole dataset from the disk.

![Dask's directed acyclic diagram (DAG)](lazy/dask_diagram.svg)

Let's load the same dataset but stored with a different chunking, in this case optimised for slicing it:

In [14]:
im = hs.load("lazy/lazy_demo_data_hyperspy_image_chunking.hspy", lazy=True)

Let's check what sort of object we've loaded

In [15]:
print(im)

<LazySignal2D, title: Stack of  (0,), dimensions: (4096|512, 512)>


It's the same dataset, but now the "view" is different: it is configured as an image stack.

We could have obtained the same view by transposing the spectrum image as follow

In [16]:
sT = s.T
print(sT)

<LazySignal2D, title: Stack of  (0,), dimensions: (4096|512, 512)>


It looks the same, but when we look at if we look at the chunks we'll see that they're very different:

In [17]:
print(sT.data)

dask.array<transpose, shape=(4096, 512, 512), dtype=int64, chunksize=(4096, 4, 4)>


For the transposed dataset the chunks are just like in the file, good for looking at spectra but bad for looking at images.

Let's have a look at the chunking of the ``lazy_demo_data_hyperspy_image_chunking.hspy`` file:

In [18]:
print(im.data)

dask.array<array, shape=(4096, 512, 512), dtype=int64, chunksize=(1, 512, 512)>


In this case, the chunks are individual images, what's optimal for slicing the spectrum image.

In [19]:
im.plot(navigator="slider")

We notice that navigation the image stack is fast, despite the fact that the data is not all loaded into memory at once but only 1 image at a time on demand.

### Creating a "navigator" for the two views of the dataset

Let's create a "navigator" for the spectrum image the way that HyperSpy usually do it, but summing all the signal dimensions.

This operation could be performed as follows for the spectrum image:

```python
s_navigator = s.sum(("x", "y"))
```

However, in this case, due to the particular chunking of this file, we know that this is a bad idea as it'll require loading the whole dataset in order to perform the computation. Instead we could sum all the navigation dimensions of the image stack which is equivalent but a lot more efficient in this case.

In [20]:
s.axes_manager[0].name = "x"
s.axes_manager[1].name = "y"

In [21]:
im

<LazySignal2D, title: Stack of  (0,), dimensions: (4096|512, 512)>

In [22]:
s_navigator = im.sum()

INFO:hyperspy._signals.lazy:Rechunking.
Original chunks: ((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Notice the log messages informing us that HyperSpy has automatically rechunked the dataset. This is in order to optimize the chunks for the operation requested. However, this rechunking is "virtual" i.e. HyperSpy obviously didn't modify the chunks in the file. This is still useful to define the size of the chunks that will be processed by the ``sum`` function.

Notice also that performing the sum took no time. That's because so far it hasn't actually performed the operation. We can confirm this by verifying that the object is a ``LazySignal``

In [23]:
print(s_navigator)

<LazySignal2D, title: Stack of  (0,), dimensions: (|512, 512)>


In order to compute the sum we have to call the ``compute`` method.

In [24]:
s_navigator.compute()

[########################################] | 100% Completed | 35.5s


In [ ]:
print(s_navigator)

Now ``s_navigator`` is no longer a ``LazySignal``.

We can generate a navigator for the image stack in the same way:

In [25]:
im_navigator = s.sum()
im_navigator.compute()

INFO:hyperspy._signals.lazy:Rechunking.
Original chunks: ((4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4), (4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4), (4096,))
INFO:hyperspy._signals.lazy:Final chunks: ((64, 64, 64, 64, 64, 64, 64, 64), (64, 64, 64, 64, 64, 64, 64, 64), (4096,)) 


[########################################] | 100% Completed | 41.5s


Let's now use the navigators to navigate the spectrum image and image stack

In [27]:
s.plot(navigator=s_navigator)

In [ ]:
im.plot(navigator=im_navigator)

## ROI in navigation dimension

In [28]:
s.plot(navigator=s_navigator)

Let's create a rectagular ROI

In [29]:
s_roi = hs.roi.RectangularROI(left=100, top=100, right=110, bottom=110)

In [30]:
s_roi

RectangularROI(left=100, top=100, right=110, bottom=110)

In [31]:
print(s_roi)

RectangularROI(left=100, top=100, right=110, bottom=110)


Calling this object extracts a rectangular roi from a HyperSpy signal. For example:

In [34]:
print(s_roi(s))

<LazySignal1D, title: Stack of  (0,), dimensions: (9, 10|4096)>


We can change its parameters interactively displaying the associated widget as follows:

In [33]:
s_roi.gui()

In [ ]:
print(s_roi(s))

And also by adding a widget to the navigator plot

In [35]:
w = s_roi.add_widget(s)

So far, we have to execute manually the ROI on the spectrum image every time that we change its parameters. We can perform this operation automatically instead:

In [36]:
s2 = hs.interactive(s_roi, signal=s,
                    event=s_roi.events.changed,
                    recompute_out_event=None)

In [37]:
print(s2)

<LazySignal1D, title: Stack of  (0,), dimensions: (9, 10|4096)>


We can use the ``interactive`` function to compute the sum of the extracted spectrum image interactively as follows:

In [38]:
s2_sum = hs.interactive(s2.sum, rechunk=False,
                        event=s2.events.data_changed,
                        recompute_out_event=None)

In [39]:
s2_sum

<LazySignal1D, title: Stack of  (0,), dimensions: (|4096)>

And plot it. Notice that the spectrum should update when we change the ROI parameters

In [40]:
s2_sum.plot()

What if we want to integrate the peaks in the spectra and display the corresponding image?

Whe working with in-memory data we could do it using the spectrum image but, in lazy mode, it is better to work with the image stack version of the dataset.

In [41]:
im.plot(navigator=im_navigator)

In [ ]:
im_roi = hs.roi.SpanROI(100, 200)
im_w = im_roi.add_widget(im)

In [ ]:
im_roi.gui()

In [ ]:
im2 = hs.interactive(im_roi, signal=im,
                     event=im_roi.events.changed,
                     recompute_out_event=None)

In [ ]:
im2_sum = hs.interactive(im2.sum, rechunk=False,
                         event=im2.events.data_changed,
                         recompute_out_event=None)

In [ ]:
im2_sum.plot()

## Rebinning

In [46]:
s = hs.load("lazy/lazy_demo_data_hyperspy_image_chunking.hspy", lazy=True)

In [43]:
print(s.data)

dask.array<array, shape=(4096, 512, 512), dtype=int64, chunksize=(1, 512, 512)>


In [48]:
s._make_lazy(rechunk="dask_auto")

INFO:hyperspy._signals.lazy:Rechunking.
Original chunks: ((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
print(s.data)

In [50]:
srb = s.rebin((4096, 128, 128))

INFO:hyperspy._signals.lazy:Rechunking.
Original chunks: ((64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64), (512,), (512,))
INFO:hyperspy._signals.lazy:Final chunks: ((128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128), (512,), (128, 128, 128, 128)) 


In [51]:
s

<LazySignal2D, title: Stack of  (0,), dimensions: (4096|512, 512)>

In [52]:
srb.compute()

[########################################] | 100% Completed | 38.8s


In [53]:
srb.plot()